#### Задача 1.

Возьмите любой достаточно длинный новостной текст, извлеките из него все именованные сущности и нормализуйте их.

In [ ]:
!pip install natasha

In [26]:
from natasha import (
    Segmenter,
    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    Doc,
    NewsNERTagger,
    MorphVocab
)

In [32]:
text = """  Москва. 3 января. INTERFAX.RU - После крушения танкеров с нефтепродуктами с пляжей черноморского побережья Краснодарского края собрано более 86 тыс. тонн загрязненного мазутом грунта, сообщил в своем телеграм-канале губернатор региона Вениамин Кондратьев.

"Команды муниципалитетов края, волонтеры, сотрудники МЧС и спасательных служб в общей сложности собрали уже 86 тысяч тонн замазученного грунта, за последние сутки - 8 тысяч тонн", - написал он.

Кондратьев отметил, что на площадку временного хранения в хуторе Воскресенском за все время вывезено около 51 тыс. тонн загрязненного песка и грунта. "Оттуда песок, перемешанный с мазутом, отправляем на утилизацию на специализированные предприятия", - написал губернатор.
15 декабря в Керченском проливе потерпели крушение танкеры с мазутом "Волгонефть 212" и "Волгонефть 239", произошел разлив топлива. Нефтепродукты вынесло на берег, загрязнение растянулось более чем на 60 км.

К ликвидации ЧС привлечены, с учетом волонтеров, более 10 тыс. человек и 700 единиц техники.

По последним данным Минтранса РФ, при крушении танкеров были повреждены четыре танка с нефтепродуктами, в море попало около 2,4 тыс. тонн мазута.

25 декабря губернатор Краснодарского края Вениамин Кондратьев ввел режим ЧС на территории региона. 26 декабря правительственная комиссия под председательством главы МЧС РФ Александра Куренкова приняла решение о введении ЧС федерального уровня. """

In [33]:
segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)

doc = Doc(text)

doc.segment(segmenter)
doc.tag_morph(morph_tagger)
doc.parse_syntax(syntax_parser)

In [34]:
ner_tagger = NewsNERTagger(emb)
doc.tag_ner(ner_tagger)
doc.ner.print()

  Москва. 3 января. INTERFAX.RU - После крушения танкеров с 
  LOC───            ORG────────                             
нефтепродуктами с пляжей черноморского побережья Краснодарского края 
                                                 LOC──────────────── 
собрано более 86 тыс. тонн загрязненного мазутом грунта, сообщил в 
своем телеграм-канале губернатор региона Вениамин Кондратьев.
                                         PER──────────────── 
"Команды муниципалитетов края, волонтеры, сотрудники МЧС и 
                                                     ORG   
спасательных служб в общей сложности собрали уже 86 тысяч тонн 
замазученного грунта, за последние сутки - 8 тысяч тонн", - написал 
он.
Кондратьев отметил, что на площадку временного хранения в хуторе 
PER───────                                                       
Воскресенском за все время вывезено около 51 тыс. тонн загрязненного 
LOC──────────                                                        
песка и грунта. "

In [35]:
morph_vocab = MorphVocab()

for span in doc.spans:
    span.normalize(morph_vocab)

{_.text: _.normal for _ in doc.spans}

{'Москва': 'Москва',
 'INTERFAX.RU': 'INTERFAX.RU',
 'Краснодарского края': 'Краснодарский край',
 'Вениамин Кондратьев': 'Вениамин Кондратьев',
 'МЧС': 'МЧС',
 'Кондратьев': 'Кондратьев',
 'Воскресенском': 'Воскресенский',
 'Керченском проливе': 'Керченский пролив',
 'Минтранса': 'Минтранс',
 'РФ': 'РФ',
 'Александра Куренкова': 'Александр Куренков'}

#### Задача 2.

Постройте дерево зависимостей для одного и того же предложения в natasha и spacy или corpy (для последнего придется вручную записать файл .conll и отправить его содержимое в арборатор, например). Сравните, что вам больше нравится.

In [41]:
import spacy
from spacy import displacy

In [ ]:
!python -m spacy download ru_core_news_sm

In [51]:
nlp = spacy.load('ru_core_news_sm')
doc = nlp('Кондратьев отметил, что на площадку временного хранения в хуторе Воскресенском за все время вывезено около 51 тыс. тонн загрязненного песка и грунта.')

In [52]:
# дерево зависимостей
displacy.render(doc, style="dep")

In [53]:
for token in doc:
    print(f"{token.text:<12} {token.dep_:<10} {token.head.text}")

Кондратьев   nsubj      отметил
отметил      ROOT       отметил
,            punct      вывезено
что          mark       вывезено
на           case       площадку
площадку     obl        вывезено
временного   amod       хранения
хранения     nmod       площадку
в            case       хуторе
хуторе       nmod       площадку
Воскресенском appos      хуторе
за           case       время
все          det        время
время        obl        вывезено
вывезено     ccomp      отметил
около        case       тонн
51           nummod     тыс.
тыс.         nummod     тонн
тонн         nsubj:pass вывезено
загрязненного amod       песка
песка        nmod       тонн
и            cc         грунта
грунта       conj       песка
.            punct      отметил


In [54]:
# natasha
my_text = 'Кондратьев отметил, что на площадку временного хранения в хуторе Воскресенском за все время вывезено около 51 тыс. тонн загрязненного песка и грунта.'

In [55]:
doc_n = Doc(my_text)
doc_n.segment(segmenter)
doc_n.tag_morph(morph_tagger)
doc_n.parse_syntax(syntax_parser)

for sent in doc_n.sents:
    sent.syntax.print()

              ┌► Кондратьев    nsubj
┌─┌───────────└─ отметил       
│ │ ┌──────────► ,             punct
│ │ │ ┌────────► что           mark
│ │ │ │       ┌► на            case
│ │ │ │ ┌──►┌─└─ площадку      obl
│ │ │ │ │   │ ┌► временного    amod
│ │ │ │ │ ┌─└►└─ хранения      nmod
│ │ │ │ │ │   ┌► в             case
│ │ │ │ │ └►┌─└─ хуторе        nmod
│ │ │ │ │   └──► Воскресенском appos
│ │ │ │ │   ┌──► за            case
│ │ │ │ │   │ ┌► все           det
│ │ │ │ │   └─└─ время         obl
│ └►└─└─└─┌─└─── вывезено      ccomp
│       │ │ ┌──► около         case
│       │ │ │ ┌► 51            nummod
│       │ └►└─└─ тыс           nsubj:pass
│       │   └──► .             punct
│       └──►┌─── тонн          nsubj:pass
│           │ ┌► загрязненного amod
│         ┌─└►└─ песка         nmod
│         │   ┌► и             cc
│         └──►└─ грунта        conj
└──────────────► .             punct


In [ ]:
# ну вроде оба норм

#### Задача 3.

Возьмите пару достаточно сложных (с частеречной омонимией, например) предложений на любом языке и сделайте их полный морфосинтаксический разбор с помощью хотя бы двух любых известных вам (и работающих для этого языка) инструментов. Если эти инструменты делают разбор в одном формате (например, UD), выведите автоматически разницу в разборах.

In [ ]:
!pip install stanza

In [4]:
import stanza
import spacy

In [1]:
sent_1 = 'Подумаешь, какой он умный.'
sent_2 = 'Ты подумаешь над этим вопросом.'

In [ ]:
stanza_nlp = stanza.Pipeline(lang='ru', processors='tokenize,pos,lemma,depparse')
spacy_nlp = spacy.load("ru_core_news_sm")

In [13]:
# spacy
def my_spacy(nlp, sentence):
    doc = nlp(sentence)
    result = []
    for token in doc:
        result.append({
            "word": token.text,
            "lemma": token.lemma_,
            "pos": token.pos_,
            "morph": token.morph.to_dict(),
            "dep": token.dep_,
            "head": token.head.text,
        })
    return result

In [14]:
# stanza
def my_stanza(nlp, sentence):
    doc = nlp(sentence)
    result = []
    for sent in doc.sentences:
        for word in sent.words:
            result.append({
                "word": word.text,
                "lemma": word.lemma,
                "upos": word.upos,
                "xpos": word.xpos,
                "feats": word.feats,
                "head": word.head,
                "deprel": word.deprel,
            })
    return result

In [15]:
print(my_stanza(stanza_nlp, sent_1))
print(my_spacy(spacy_nlp, sent_1))

[{'word': 'Подумаешь', 'lemma': 'подумать', 'upos': 'VERB', 'xpos': None, 'feats': 'Aspect=Perf|Mood=Ind|Number=Sing|Person=2|Tense=Fut|VerbForm=Fin|Voice=Act', 'head': 0, 'deprel': 'root'}, {'word': ',', 'lemma': ',', 'upos': 'PUNCT', 'xpos': None, 'feats': None, 'head': 5, 'deprel': 'punct'}, {'word': 'какой', 'lemma': 'какой', 'upos': 'DET', 'xpos': None, 'feats': 'Case=Nom|Gender=Masc|Number=Sing|PronType=Int,Rel', 'head': 5, 'deprel': 'det'}, {'word': 'он', 'lemma': 'он', 'upos': 'PRON', 'xpos': None, 'feats': 'Case=Nom|Gender=Masc|Number=Sing|Person=3|PronType=Prs', 'head': 5, 'deprel': 'nsubj'}, {'word': 'умный', 'lemma': 'умный', 'upos': 'ADJ', 'xpos': None, 'feats': 'Case=Nom|Degree=Pos|Gender=Masc|Number=Sing', 'head': 1, 'deprel': 'ccomp'}, {'word': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': None, 'feats': None, 'head': 1, 'deprel': 'punct'}]
[{'word': 'Подумаешь', 'lemma': 'подумаешь', 'pos': 'VERB', 'morph': {'Aspect': 'Imp', 'Mood': 'Ind', 'Number': 'Sing', 'Person': 'Se

In [16]:
print(my_stanza(stanza_nlp, sent_2))
print(my_spacy(spacy_nlp, sent_2))

[{'word': 'Ты', 'lemma': 'ты', 'upos': 'PRON', 'xpos': None, 'feats': 'Case=Nom|Number=Sing|Person=2|PronType=Prs', 'head': 2, 'deprel': 'nsubj'}, {'word': 'подумаешь', 'lemma': 'подумать', 'upos': 'VERB', 'xpos': None, 'feats': 'Aspect=Perf|Mood=Ind|Number=Sing|Person=2|Tense=Fut|VerbForm=Fin|Voice=Act', 'head': 0, 'deprel': 'root'}, {'word': 'над', 'lemma': 'над', 'upos': 'ADP', 'xpos': None, 'feats': None, 'head': 5, 'deprel': 'case'}, {'word': 'этим', 'lemma': 'этот', 'upos': 'DET', 'xpos': None, 'feats': 'Case=Ins|Gender=Masc|Number=Sing|PronType=Dem', 'head': 5, 'deprel': 'det'}, {'word': 'вопросом', 'lemma': 'вопрос', 'upos': 'NOUN', 'xpos': None, 'feats': 'Animacy=Inan|Case=Ins|Gender=Masc|Number=Sing', 'head': 2, 'deprel': 'obl'}, {'word': '.', 'lemma': '.', 'upos': 'PUNCT', 'xpos': None, 'feats': None, 'head': 2, 'deprel': 'punct'}]
[{'word': 'Ты', 'lemma': 'ты', 'pos': 'PRON', 'morph': {'Case': 'Nom', 'Number': 'Sing', 'Person': 'Second'}, 'dep': 'nsubj', 'head': 'подумаешь'